In [ ]:
from IPython.core.display import display, HTML
import sys
display(HTML("<style>.container { width:95% !important; }</style>"))
print(sys.version)

In [ ]:
from datetime import date, datetime
import numpy as np
import pandas as pd
import dataframe_image as dfi
# pd.options.display.float_format = '{:,.3f}'.format
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns
sns.set_theme()
sns.set_color_codes()
#pd.set_option('display.max_rows', 300)
#pd.set_option('display.max_columns', 20)

# column_names = ['# Seen', 'ALSA', '# Picked', 'ATA', '# GP', 'GP WR', '# OH', 'OH WR', '# GD', 'GD WR', '# GIH', 'GIH WR', '# GND', 'GND WR', 'IWD', 'Color', 'Rarity']

from Utilities import Logger, Fetcher
import WUBRG
from WUBRG import get_color_identity, ColorSortStyles
from game_metadata import SETS, FORMATS
from game_metadata import Card,CardManager, SetMetadata, FormatMetadata
from data_fetching import DataLoader, LoadedData, DataFramer, FramedData, SetManager, CentralManager
from data_fetching.utils import get_next_17lands_update_time, get_prev_17lands_update_time, get_name_slice, get_color_slice, get_date_slice
from data_fetching.utils import rarity_filter, cmc_filter, card_color_filter, cast_color_filter, compose_filters


TRGT_SET = 'DMU'
LOG_LEVEL = Logger.FLG.DEFAULT
LOG_LEVEL

# Initialization

In [ ]:
data_manager = None
set_data = None
print(f'Available sets: {SETS}')
print(f'Target set: {TRGT_SET}')

In [ ]:
print(f"Current Local Time:  {datetime.now()}")
print(f"Last 17Lands Update: {get_prev_17lands_update_time()}")
print(f"Current UTC Time:    {datetime.utcnow()}")
print(f"Next 17Lands Update: {get_next_17lands_update_time()}")

In [ ]:
start = datetime.utcnow()

if set_data is None:
    set_data = SetManager(TRGT_SET)
set_data.check_for_updates()
card_dict = set_data.SET_METADATA.CARD_DICT

end = datetime.utcnow()
print(f"\n --- Data loaded in {end - start}.")

## Tier List Analysis

### Functions

In [ ]:
tier_to_rank = {
    "A+": 12,
    "A": 11,
    "A-": 10,
    "B+": 9,
    "B": 8,
    "B-": 7,
    "C+": 6,
    "C": 5,
    "C-": 4,
    "D+": 3,
    "D": 2,
    "D-": 1,
    "F": 0,
    "SB": None,
    "TBD": None
}

rank_to_tier = {v: k for k, v in tier_to_rank.items()}

In [ ]:
def fetch_raw_data(url):
    if url.startswith("https://www.17lands.com/tier_list/"):
        guid = url.replace("https://www.17lands.com/tier_list/", "")
        url = f"https://www.17lands.com/card_tiers/data/{guid}"
    fetcher = Fetcher()
    raw_data = fetcher.fetch(url)
    return raw_data


def parse_raw_data(raw_data):
    data = dict()
    for card_rating in raw_data:
        name = CardManager.from_name(card_rating['name']).NAME
        
        data[name] = {
            'card': card_dict[name], 
            'tier': card_rating['tier'], 
            'rank': tier_to_rank[card_rating['tier']], 
            'synergy': card_rating['flags']['synergy'], 
            'buildaround': card_rating['flags']['buildaround']
        }
    return data


def to_data_frame(data, col_name='Rank'):
    ranks = dict()
    for card in data:
        ranks[card] = data[card]['rank']

    return pd.DataFrame.from_dict(ranks, orient="index", columns=[col_name])


def frame_from_url(url, name):
    raw_data = fetch_raw_data(url)
    data = parse_raw_data(raw_data)
    frame = to_data_frame(data)
    frame.index.name = name
    
    return frame

In [ ]:
def merge_rankings(frame_list):
    frame = frame_list[0].copy(True)
    names = list()
    
    for indiv in frame_list:
        name = indiv.index.name
        names.append(name)
        frame[name] = indiv['Rank'].astype('Int64')
        
    frame = frame.drop('Rank', axis=1)
    frame['mean'] = frame.mean(axis=1)
    frame['max'] = frame.max(axis=1)
    frame['min'] = frame.min(axis=1)
    frame['range'] = frame['max'] - frame['min']
    frame['range'] = frame['range']
    frame.index.name = 'Card'
    
    # Get the difference of squares distance to figure out most 'controversial' cards.
    dist = pd.DataFrame()
    for name in names:
        dist[name] = abs(frame['mean'] - frame[name])
    dist['dist'] = dist.mean(axis=1)
    frame['dist'] = dist['dist'].round(1)

    return frame


def supplement_frame(frame, card_dict):
    links = dict()
    cast = dict()
    colors = dict()
    rarities = dict()
    cmc = dict()
    
    for card_name in card_dict:
        card = card_dict[card_name]
        links[card_name] = card.NAME
        cast[card_name] = card.CAST_IDENTITY
        colors[card_name] = card.COLOR_IDENTITY
        rarities[card_name] = card.RARITY
        cmc[card_name] = card.CMC
    
    series = frame.index.to_series()
    frame['Image'] = series.map(links)
    frame['Cast Color'] = series.map(cast)
    frame['Color'] = series.map(colors)
    frame['Rarity'] = series.map(rarities)
    frame['CMC'] = series.map(cmc)
    
    cols = list(frame.columns)
    frame = frame[['Image', 'CMC', 'Rarity', 'Color', 'Cast Color'] + cols[:-5]]

    return frame

In [ ]:
def style_frame(sub_frame):
    def hoverable(card_name):
        card = card_dict[card_name]
        hmtl = '<style>.hover_img a { position:relative; }\n' + \
        '.hover_img a span { position:absolute; display:none; z-index:300; }\n' + \
        '.hover_img a:hover span { display:block; height: 300px; width: 300px; overflow: visible; margin-left: -175px; }</style>\n' + \
        f'<div class="hover_img">\n' + \
        f'<a href="#">{card_name}<span><img src="{card.IMAGE_URL}" alt="image"/></span></a>\n' + \
        f'</div>'
        return hmtl
     
    def format_short_float(val):
        return '{:.1f}'.format(val)
    
    def format_long_float(val):
        return '{:.3f}'.format(val)
    
    sub_frame = sub_frame.style.format({
        'Image': hoverable,
        'mean': format_long_float,
        'max': format_short_float,
        'min': format_short_float,
        'range': format_short_float,
        'dist': format_short_float,
    })
    
    return sub_frame


def display_frame(frame, order='mean', filters=None):
    pd.set_option('display.max_rows', 300)
    sub_frame = frame
    
    if filters is not None:
        filt_frame = pd.DataFrame()
        for f in filters:
            filt_frame[id(f)] = f(sub_frame)
        sub_frame = sub_frame[filt_frame.T.all()]
            
    sub_frame = sub_frame.sort_values(order, ascending=False)
            
    return style_frame(sub_frame)

In [ ]:
def get_avgs(frame):
    colors = 'WUBRG'
    rarities = 'MRUC'
    avgs = dict()
    
    for r in rarities:
        color_frame = pd.DataFrame()
        
        for c in colors:    
            filt_frame = pd.DataFrame()
            filt_frame['color'] = cast_color_filter(c, ColorSortStyles.exact)(frame)
            filt_frame['rarity'] = rarity_filter(r)(frame)
            working = frame[filt_frame.T.all()].copy(True)
            working = working.drop(['min', 'max', 'range', 'dist', 'Image', 'CMC', 'Rarity', 'Color', 'Cast Color'], axis=1)
            color_frame[c] = working.mean().round(1)
            
        avgs[r] = color_frame.T
    
    ret = pd.concat(avgs)
    names = list(ret.columns)[:-1]
    
    ret['max'] = ret.max(axis=1)
    ret['min'] = ret.min(axis=1)
    ret['range'] = ret['max'] - ret['min']
    
    # Get the difference of squares distance to figure out most 'controversial' cards.
    dist = pd.DataFrame()
    for name in names:
        dist[name] = abs(ret['mean'] - ret[name])
    dist['dist'] = dist.mean(axis=1)
    ret['dist'] = dist['dist'].round(2)

    return ret

### Data Analysis

After loading all of the data, we crunch it to figure out the average grade that each contributer has given to cards. While not totally necessary, this helps anchor evaluations between contrinutors, as one person's C may be another person's C+. 

In [ ]:
pairs = [
    ("https://www.17lands.com/tier_list/03ab10d25d8841f8aef0aa90b30c434b", 'Zac'),
    ("https://www.17lands.com/tier_list/3078f70b9a0d415ebf9f555439f5aedb", 'Klarm'),
    ("https://www.17lands.com/tier_list/a1ae9a695abb4921866f7f4a9a22e21f", 'Phyrre'),
    ("https://www.17lands.com/tier_list/19ee3198a63b499bb7c25e6ceaae00ba", 'Nomad'),
    ("https://www.17lands.com/tier_list/f1b015031fd741268c5228d4d8435938", 'Catharsis'),
    ("https://www.17lands.com/tier_list/e854e4a17f5147ad9908c5f6e5d2e87c", "Rewind"),
    ("https://www.17lands.com/tier_list/1d343c3e1f0d4cfd8c356cdaa42bb329", "Davis"),
    ("https://www.17lands.com/tier_list/e12ee0b1fadc4ab7b8de4c3730878a90", "Chord")
]

indiv_frames = [frame_from_url(*pair) for pair in pairs]
base_frame = merge_rankings(indiv_frames)
frame = supplement_frame(base_frame.copy(True), card_dict)
# base_frame.to_csv("C:\\Users\\Zachary\\Downloads\\TierlistSummary.csv", encoding='utf-8')
base_frame.mean()

### 'Controversial' Cards

Here, we separate the cards by rarity, and order them by the sum of the difference between each rating and the mean. This is saved in the `dist` column, and the higher the `dist` the lesss people agree on the rating of a card. If the `dist` is zero, everyone has given the card the same grade. The value of dist can either mean that some people disagree and most agree, or everyone disagrees to varying degrees.

To me, anyting with a dist less than or equal to 0.6 is pretty agreed upon, and anything higher than 1.0 is fairly contentious.

In [ ]:
ordering = 'dist'
commons = display_frame(frame, order=ordering, filters=[rarity_filter('C')]).data
uncommons = display_frame(frame, order=ordering, filters=[rarity_filter('U')]).data
rare = display_frame(frame, order=ordering, filters=[rarity_filter('R')]).data
mythic = display_frame(frame, order=ordering, filters=[rarity_filter('M')]).data

display = pd.concat([commons, uncommons, rare, mythic])
style_frame(display)

### Top Cards

This is the short-list of top commons and uncommons, ordered by colour, based on average rating of contributors.

In [ ]:
ordering = 'mean'
top = list()

for c in 'WUBRG':
    top.append(display_frame(frame, order=ordering, filters=[rarity_filter('U'), cast_color_filter(c, ColorSortStyles.exact)]).data.head(3))
    top.append(display_frame(frame, order=ordering, filters=[rarity_filter('C'), cast_color_filter(c, ColorSortStyles.exact)]).data.head(5))

display = pd.concat(top)
style_frame(display)

This is the entire list of cards, ordered by rarity, then by average rating amongst contributers.

In [ ]:
ordering = 'mean'
commons = display_frame(frame, order=ordering, filters=[rarity_filter('C')]).data
uncommons = display_frame(frame, order=ordering, filters=[rarity_filter('U')]).data
rare = display_frame(frame, order=ordering, filters=[rarity_filter('R')]).data
mythic = display_frame(frame, order=ordering, filters=[rarity_filter('M')]).data

display = pd.concat([commons, uncommons, rare, mythic])
style_frame(display)

### Color Ratings

This is a little rougher, and more difficult to parse, but this is the avreage grade for each contirbutor given to cards of 'x' colour and 'y' rarity. it should serve as a rough estimation of how strong each person thinks a given colour is, though doesn't necessarily translate well to colour combinations.

In [ ]:
get_avgs(frame)

These are the rankings of all the non-basic lands.
In short:
- Gx taplands are highest rated
- UR tapland is slightly ahead of other non-GX tap lands
- Taplands are ahead of painlands

In [ ]:
lands = [
     'Adarkar Wastes',
     'Caves of Koilos',
     'Shivan Reef',
     'Yavimaya Coast',
     'Sulfurous Springs',
     'Karplusan Forest',
     'Plaza of Heroes',
     'Thran Portal',
     'Crystal Grotto',
     'Idyllic Beachfront',
     'Sunlit Marsh',
     'Sacred Peaks',
     'Radiant Grove',
     'Contaminated Aquifer',
     'Molten Tributary',
     'Tangled Islet',
     'Geothermal Bog',
     'Haunted Mire',
     'Wooded Ridgeline',
]
style_frame(frame.loc[lands])

### Gut-Check

The two selections below are mainly for my benefit (though I can send any contributor a personalized copy if they ask!), to check my opions of cards against what everyone else thinks. Any card I have graded more than a grade higher or lower than average is shown here, to give a more concrete idea of where _I_ disagree, than where _people_ disagree.

In [ ]:
play_frame = frame.copy(True)
play_frame['diff'] = frame['Zac'] - frame['mean']
style_frame(play_frame[play_frame['diff'] > 1].sort_values('diff', ascending=False))

In [ ]:
play_frame = frame.copy(True)
play_frame['diff'] = frame['Zac'] - frame['mean']
style_frame(play_frame[play_frame['diff'] < -1].sort_values('diff', ascending=True))

### Specific Checks

These are a list of specific cards I wanted to compare, because they either seem interesting, are cards that feel similar-but-different. Feel free to ask me to add certain cards to this list if you're curious

In [ ]:
play_frame.loc[['Love Song of Night and Day', 'Dragon Whelp', 'Soaring Drake', 'Mesa Cavalier', 'Leyline Binding', 'Prayer of Binding', 'Deathbloom Gardener', 'Salvaged Manaworker']]